# Gaussian Processes on Molecules in GPflow

This example demonstrates how it's possible to train a Gaussian Process to predict molecular properties using the GPflow library https://gpflow.readthedocs.io/en/master/ and a custom-defined Tanimoto kernel.

In our example, we'll be trying to predict the experimentally-determined electronic transition wavelengths of molecular photoswitches, a class of molecule that undergoes a reversible transformation between its E and Z isomers upon the application of light. The figure below, created by the artistically indefatigable Aditya Thawani https://twitter.com/RaymondThawani who managed to feature his own hand in the logo for this repo, illustrates this process.

<p align="center">
  <img src="../photoswitching.png" width="500" title="logo">
</p>

We'll start by importing all of the machine learning and chemistry libraries we're going to use

In [1]:
import gpflow
from gpflow.mean_functions import Constant
from gpflow.utilities import positive, print_summary
from gpflow.utilities.ops import broadcasting_elementwise
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from rdkit.Chem import AllChem, Descriptors, MolFromSmiles
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.preprocessing import StandardScaler
import tensorflow as tf

As our molecular representation, we're going to be working with the widely-used Morgan fingerprints. Under this representation, molecules are represented as bit vectors which means that standard Gaussian Process kernels such as the squared exponential kernel or the Matern$\frac{1}{2}$ kernel won't be ideal as they're designed for continuous spaces. We can however, design a custom "Tanimoto" or "Jaccard" kernel designed to compute similarity between bit vectors

<p align="center">
  <img src="../equation_tanimoto.png" width="400" title="logo">
</p>

where $\textbf{x}$ and $\textbf{x'}$ are bit vectors and $\sigma$ is the kernel variance. It is relatively straightforward to define a custom Tanimoto kernel in GPflow cf. https://gpflow.readthedocs.io/en/master/notebooks/tailor/kernel_design.html. The definition below differs slightly because it is designed to compute the Tanimoto similarity over matrix input.

In [2]:
class Tanimoto(gpflow.kernels.Kernel):
    def __init__(self):
        super().__init__()
        # We constrain the value of the kernel variance to be positive when it's being optimised
        self.variance = gpflow.Parameter(1.0, transform=positive())

    def K(self, X, X2=None):
        """
        Compute the Tanimoto kernel matrix σ² * ((<x, y>) / (||x||^2 + ||y||^2 - <x, y>))

        :param X: N x D array
        :param X2: M x D array. If None, compute the N x N kernel matrix for X.
        :return: The kernel matrix of dimension N x M
        """
        if X2 is None:
            X2 = X

        Xs = tf.reduce_sum(tf.square(X), axis=-1)  # Squared L2-norm of X
        X2s = tf.reduce_sum(tf.square(X2), axis=-1)  # Squared L2-norm of X2
        outer_product = tf.tensordot(X, X2, [[-1], [-1]])  # outer product of the matrices X and X2

        # Analogue of denominator in Tanimoto formula

        denominator = -outer_product + broadcasting_elementwise(tf.add, Xs, X2s)

        return self.variance * outer_product/denominator

    def K_diag(self, X):
        """
        Compute the diagonal of the N x N kernel matrix of X
        :param X: N x D array
        :return: N x 1 array
        """
        return tf.fill(tf.shape(X)[:-1], tf.squeeze(self.variance))


Next, we read in our photoswitch molecules, represented as SMILES. The property we're predicting is the electronic transition wavelength of the E isomer of each molecule.

In [3]:
df = pd.read_csv('../dataset/photoswitches.csv')  # Load the photoswitch dataset using pandas

# Create a list of molecules smiles and associated properties
smiles_list = df['SMILES'].to_list()
property_vals = df['E isomer pi-pi* wavelength in nm'].to_numpy()

# Delete NaN values
smiles_list = list(np.delete(np.array(smiles_list), np.argwhere(np.isnan(property_vals))))
y = np.delete(property_vals, np.argwhere(np.isnan(property_vals)))

We convert our molecules to 2048 bit Morgan fingerprints using a bond radius of 3. We denote the molecules as X and the property values by y. I guess this might make it easier to remember what the inputs and outputs are!

In [4]:
rdkit_mols = [MolFromSmiles(smiles) for smiles in smiles_list]
X = [AllChem.GetMorganFingerprintAsBitVect(mol, radius=3, nBits=2048) for mol in rdkit_mols]
X = np.asarray(X)

We define our optimisation objective for fitting the GP hyperparameters, namely the negative log marginal likelihood.

In [5]:
# We define the Gaussian Process Regression Model using the Tanimoto kernel

m = None

def objective_closure():
    return -m.log_marginal_likelihood()

We define a utility function to standardise our output values. This is typically good practice before fitting a GP cf the "Other Tricks" slide in Iain Murray's presentation https://homepages.inf.ed.ac.uk/imurray2/teaching/08gp_slides.pdf

In [6]:
def transform_data(X_train, y_train, X_test, y_test):
    """
    Apply feature scaling, dimensionality reduction to the data. Return the standardised and low-dimensional train and
    test sets together with the scaler object for the target values.

    :param X_train: input train data
    :param y_train: train labels
    :param X_test: input test data
    :param y_test: test labels
    :return: X_train_scaled, y_train_scaled, X_test_scaled, y_test_scaled, y_scaler
    """

    x_scaler = StandardScaler()
    X_train_scaled = x_scaler.fit_transform(X_train)
    X_test_scaled = x_scaler.transform(X_test)
    y_scaler = StandardScaler()
    y_train_scaled = y_scaler.fit_transform(y_train)
    y_test_scaled = y_scaler.transform(y_test)

    return X_train_scaled, y_train_scaled, X_test_scaled, y_test_scaled, y_scaler

We define an 80/20 train/test split

In [7]:
test_set_size = 0.2

Next we fit the GP. We can inspect the learned kernel hyperparameters although these might not be so intuitive in the case of bit vectors!

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_set_size, random_state=0)

y_train = y_train.reshape(-1, 1)
y_test = y_test.reshape(-1, 1)

#  We standardise the outputs but leave the inputs unchanged

_, y_train, _, y_test, y_scaler = transform_data(X_train, y_train, X_test, y_test)

X_train = X_train.astype(np.float64)
X_test = X_test.astype(np.float64)

k = Tanimoto()
m = gpflow.models.GPR(data=(X_train, y_train), mean_function=Constant(np.mean(y_train)), kernel=k, noise_variance=1)

# Optimise the kernel variance and noise level by the marginal likelihood

opt = gpflow.optimizers.Scipy()
opt.minimize(objective_closure, m.trainable_variables, options=dict(maxiter=100))
print_summary(m)

# mean and variance GP prediction

y_pred, y_var = m.predict_f(X_test)
y_pred = y_scaler.inverse_transform(y_pred)
y_test = y_scaler.inverse_transform(y_test)

╒═════════════════════════╤═══════════╤══════════════════╤═════════╤═════════════╤═════════╤═════════╤═══════════╕
│ name                    │ class     │ transform        │ prior   │ trainable   │ shape   │ dtype   │     value │
╞═════════════════════════╪═══════════╪══════════════════╪═════════╪═════════════╪═════════╪═════════╪═══════════╡
│ GPR.mean_function.c     │ Parameter │                  │         │ True        │ ()      │ float64 │ 0.0443125 │
├─────────────────────────┼───────────┼──────────────────┼─────────┼─────────────┼─────────┼─────────┼───────────┤
│ GPR.kernel.variance     │ Parameter │ Softplus         │         │ True        │ ()      │ float64 │ 0.463848  │
├─────────────────────────┼───────────┼──────────────────┼─────────┼─────────────┼─────────┼─────────┼───────────┤
│ GPR.likelihood.variance │ Parameter │ Softplus + Shift │         │ True        │ ()      │ float64 │ 0.0129625 │
╘═════════════════════════╧═══════════╧══════════════════╧═════════╧════════════

Now we can output the train and test root-mean-square error (RMSE), mean absolute error (MAE) and $R^2$ value. 

In [9]:
# Output Standardised RMSE and RMSE on Train Set

y_pred_train, _ = m.predict_f(X_train)
train_rmse_stan = np.sqrt(mean_squared_error(y_train, y_pred_train))
train_rmse = np.sqrt(mean_squared_error(y_scaler.inverse_transform(y_train), y_scaler.inverse_transform(y_pred_train)))
print("\nTrain RMSE (Standardised): {:.3f} nm".format(train_rmse_stan))
print("Train RMSE: {:.3f} nm".format(train_rmse))


# Output R^2, RMSE and MAE on the test set
score = r2_score(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
mae = mean_absolute_error(y_test, y_pred)

print("\nTest R^2: {:.3f}".format(score))
print("Test RMSE: {:.3f} nm".format(rmse))
print("Test MAE: {:.3f} nm".format(mae))


Train RMSE (Standardised): 0.036 nm
Train RMSE: 2.422 nm

Test R^2: 0.916
Test RMSE: 17.997 nm
Test MAE: 11.333 nm


Not bad! In our paper, we compared the predictions of the GP-Tanimoto model against those made by a cohort of human photoswitch chemists achieving lower test error in the case of all 5 molecules comprising the test set:

<p align="center">
  <img src="../humanGraph.png" width="500" title="logo">
</p>

MAEs in this case are computed on a per molecule basis across all human participants.

One defining characteristics of GPs is their ability to produce calibrated uncertainty estimates. In practical terms these uncertainty estimates can be helpful in capturing the model's confidence; if the test molecule is very different from the molecules seen in training the model can tell you! 

In programmatic terms, the test set uncertainties can be accessed by inspecting the variable y_var. One can obtain a ranked confidence list for the predictions by e.g.

In [10]:
ranked_confidence_list = np.argsort(y_var, axis=0).flatten()
print(ranked_confidence_list)

[14 56 72 24 19  6 70 27 55 46 67 45 61  3 71 25 22 17 34  5 31 20 12 37
 76  9 32 44 62 48 41 29 36 15 28 59 68 60  4  2 30 16  7 73 75 58 38 64
 21 77 69 18 10 47  0 65 63 26 35 33  1 42 54 78 53 52 57 43 49 74 13 23
 40 39 51 66  8 11 50]


Which outputs a list in which the molecules (represented by their test set index) are ranked by their prediction confidenceThe predict_with_GPR.py script offers further details. 

Graphically, it is possible to generate confidence-error curves in order to check that the uncertainties obtained by the GP are actually correlated with test set error.

<p align="center">
  <img src="../confidence_curve.png" width="500" title="logo">
</p>

In the plot above the x-axis, confidence percentile, may be obtained simply by ranking each model prediction of the test set in terms of the predictive variance at the location of that test input. As an example, molecules that lie in the 80th confidence percentile will be the 20% of test set molecules with the lowest model uncertainty. We then measure the prediction error at each confidence percentile across 200 random train/test splits to see whether the model’s confidence is correlated with the prediction error. We observe that the GP-Tanimoto model’s uncertainty estimates are positively correlated with prediction error.

The practical takeaway from this plot is a proof of concept that model uncertainty can be incorporated into the decision process for selecting which photoswitch molecules to physically synthesise in the lab; if the predicted wavelength value of an "unsynthesised" molecule is desirable and the confidence in this prediction is high you might want to make it!

Another nice benefit of GPs; it is  possible to interpret the chemistry which different molecular representations capture by inspecting the learned kernel. See our paper for more details! https://chemrxiv.org/articles/preprint/The_Photoswitch_Dataset_A_Molecular_Machine_Learning_Benchmark_for_the_Advancement_of_Synthetic_Chemistry/12609899

<p align="center">
  <img src="../triangleN.png" width="500" title="logo">
</p>

If you find this dataset or implementation please consider citing our paper:

```
@article{Thawani2020,
author = "Aditya Thawani and Ryan-Rhys Griffiths and Arian Jamasb and Anthony Bourached and Penelope Jones and William McCorkindale and Alexander Aldrick and Alpha Lee",
title = "{The Photoswitch Dataset: A Molecular Machine Learning Benchmark for the Advancement of Synthetic Chemistry}",
year = "2020",
month = "7",
url = "https://chemrxiv.org/articles/preprint/The_Photoswitch_Dataset_A_Molecular_Machine_Learning_Benchmark_for_the_Advancement_of_Synthetic_Chemistry/12609899",
doi = "10.26434/chemrxiv.12609899.v1"
}
```

As well as GPflow

```
@article{GPflow2020multioutput,
  author = {{van der Wilk}, Mark and Dutordoir, Vincent and John, ST and
            Artemev, Artem and Adam, Vincent and Hensman, James},
  title = {A Framework for Interdomain and Multioutput {G}aussian Processes},
  year = {2020},
  journal = {arXiv:2003.01115},
  url = {https://arxiv.org/abs/2003.01115}
}
```

and the originators of the Tanimoto kernel:

```
@article{ralaivola2005graph,
  title={Graph kernels for chemical informatics},
  author={Ralaivola, Liva and Swamidass, Sanjay J and Saigo, Hiroto and Baldi, Pierre},
  journal={Neural networks},
  volume={18},
  number={8},
  pages={1093--1110},
  year={2005},
  publisher={Elsevier}
}
```